# Converting County GeoJSON into Regular Bounding Box

## Part 1: Introduction

This Jupyter Notebook is intended to create county bounding box based on county GeoJSON/Shapefile.

## Part 2: Preparation

We will be using **Jupyter Notebook(anaconda 3)** to edit and run the script. Information on Anaconda installation can be found <a href='https://docs.anaconda.com/anaconda/install/'>here</a>. Please note that this script is running on Python 3.

***Usually, You can download county boundary GeoJSON file from state data portal, which typically is ArcGIS Hub.***

To run this script you need:
- county GeoJSON/Shapefile stored in **state** folder
- directory path (**geojson** folder > **state** folder)

The script currently prints one GeoJSON file:
- **state_bbox**.json

>Original created on Jan 31 2021<br>
@author: Yijing Zhou @YijingZhou33

## Part 3: Get Started

###  Step 1: Import modules

In [1]:
import pandas as pd
import os
import geopandas as gpd
import json
from itertools import chain
import string
import folium

### Step 2: Manual items to change

In [4]:
state = 'Ohio'

###### Rawdata is Shapefile ######
# rawdata = ''

###### Rawdata is GeoJSON ######
rawdata = 'ODOT_County_Boundaries'

## Part 4: Build up county GeoJSON schema

###  Step 3: Convert GeoJSON/Shapefile to JSON

In [5]:
output = os.path.join('geojson', state, state + '_bbox.json')

def conversion(inputfile):
    ## convert file to json 
    inputfile = json.loads(inputfile.to_json())
    ## display features properties as dataframe
    df = pd.json_normalize(inputfile['features'])
    return df

###### Rawdata is Shapefile ######

# **************** uncomment **********************
# shp = os.path.join('geojson', state, rawdata)
# county_shp = gpd.read_file(shp, driver = 'shapefile').to_crs('EPSG:4326')
# df = conversion(county_shp)
# *************************************************

###### Rawdata is GeoJSON ######

# **************** uncomment **********************
geojson = os.path.join('geojson', state, rawdata + '.geojson')
county_geojson = gpd.read_file(geojson).to_crs('EPSG:4326')
df = conversion(county_geojson)
# *************************************************
df

,id,type,properties.AREA_ID,properties.AREA_SQMI,properties.COUNTY_CD,properties.COUNTY_SEA,properties.ELEVATION1,properties.ELEVATION_,properties.FIPS_COUNT,properties.LAT_NORTH_,...,properties.POP_2010,properties.SHAPE_STAr,properties.SHAPE_STLe,properties.STATE_PLAN,properties.created_da,properties.created_us,properties.last_edi_1,properties.last_edite,geometry.type,geometry.coordinates
0,0,Feature,53,557.74,HIG,HILLSBORO,700,1340,39071,39.380,...,43589,2.407747e+09,213157.740992,S,2015-08-11T00:00:00,Esri_Anonymous,2015-08-11T00:00:00,Esri_Anonymous,Polygon,"[[[-83.7832961536684, 39.263819107361286], [-8..."
1,1,Feature,54,423.50,HOC,LOGAN,640,1220,39073,39.662,...,29380,1.844277e+09,215451.632241,S,2015-08-11T00:00:00,Esri_Anonymous,2015-08-11T00:00:00,Esri_Anonymous,Polygon,"[[[-82.49594955725306, 39.60265262918404], [-8..."
2,2,Feature,55,424.03,HOL,MILLERSBURG,800,1380,39075,40.669,...,42366,1.905310e+09,193045.013152,N,2015-08-11T00:00:00,Esri_Anonymous,2015-08-11T00:00:00,Esri_Anonymous,Polygon,"[[[-81.87726736991834, 40.66712573154221], [-8..."
3,3,Feature,56,495.96,HUR,NORWALK,600,1200,39077,41.291,...,59626,2.267372e+09,197349.121399,N,2015-08-11T00:00:00,Esri_Anonymous,2015-08-11T00:00:00,Esri_Anonymous,Polygon,"[[[-82.83546635698313, 41.144068750366614], [-..."
4,4,Feature,143,543.97,FRA,COLUMBUS,670,1130,39049,40.144,...,1163414,2.401407e+09,211754.151636,S,2015-08-11T00:00:00,Esri_Anonymous,2015-08-11T00:00:00,Esri_Anonymous,Polygon,"[[[-83.24595801241205, 39.96573664052778], [-8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,83,Feature,28,414.92,VIN,MCARTHUR,580,1140,39163,39.413,...,13435,1.794678e+09,212461.296202,S,2015-08-11T00:00:00,Esri_Anonymous,2015-08-11T00:00:00,Esri_Anonymous,Polygon,"[[[-82.51709903612551, 39.369419160060446], [-..."
84,84,Feature,29,407.21,WAR,LEBANON,570,1060,39165,39.590,...,212693,1.770899e+09,178414.982467,S,2015-08-11T00:00:00,Esri_Anonymous,2015-08-11T00:00:00,Esri_Anonymous,Polygon,"[[[-83.98858711373614, 39.44441360469584], [-8..."
85,85,Feature,51,411.07,HAS,CADIZ,860,1360,39067,40.434,...,15864,1.829951e+09,181149.223466,N,2015-08-11T00:00:00,Esri_Anonymous,2015-08-11T00:00:00,Esri_Anonymous,Polygon,"[[[-81.27272594697756, 40.369234906974945], [-..."
86,86,Feature,77,506.13,PIC,CIRCLEVILLE,630,1090,39129,39.813,...,55698,2.214650e+09,215128.493147,S,2015-08-11T00:00:00,Esri_Anonymous,2015-08-11T00:00:00,Esri_Anonymous,Polygon,"[[[-82.8430402189827, 39.56150011538731], [-82..."


### Step 4: Clean up and format columns

In [6]:
def rename(df):
    ## possible county column names in the dataframe
    clist = ['properties.COUNTY', 'properties.NAME', 'properties.COUNTY_NAM', 'properties.COUNTY_NAME', 'properties.CTY_NAME']
    
    if set(df.columns).intersection(set(clist)):
        cname = ''.join(set(df.columns).intersection(set(clist)))
    else:
        cname = input('Please enter the column storing county names: ').strip()
          
    df = df[[cname, 'geometry.coordinates']].rename(
            columns={cname:'County', 'geometry.coordinates':'boundingBox'})      
    ## capitalize the first letter of each word in the county name
    df['County'] = df['County'].apply(lambda row: string.capwords(row))
    df['State'] = state
    
    return df

df = rename(df)

Please enter the column storing county names: properties.COUNTY_SEA


###  Step 5: Create bounding box

In [ ]:
def bbox(points):
    x_coordinates, y_coordinates = zip(*points)
    return ','.join(str(x) for x in [min(x_coordinates), min(y_coordinates), max(x_coordinates), max(y_coordinates)])

def coordinates_bbox(df):
    for _, row in df.iterrows():
        ## geometry is Polygon
        if type(row['boundingBox'][0][0][0]) is float:
            row['boundingBox'] = bbox(row['boundingBox'][0])
        else:
        ## geometry is Multipolygon
            row['boundingBox'] = bbox(list(chain.from_iterable([l[0] for l in row['boundingBox']])))
            
coordinates_bbox(df)

###  Step 6: Round coordinates to 2 decimal places

In [ ]:
## create regular bouding box coordinate pairs and round them to 2 decimal places
df = pd.concat([df, df['boundingBox'].str.split(',', expand=True).astype(float).round(2)], axis=1).rename(
    columns={0:'minX', 1:'minY', 2:'maxX', 3:'maxY'})
df['maxXmaxY'] = df.apply(lambda row: [row.maxX, row.maxY], axis = 1)
df['maxXminY'] = df.apply(lambda row: [row.maxX, row.minY], axis = 1)
df['minXminY'] = df.apply(lambda row: [row.minX, row.minY], axis = 1)
df['minXmaxY'] = df.apply(lambda row: [row.minX, row.maxY], axis = 1)
df['Coordinates'] = df[['maxXmaxY', 'maxXminY', 'minXminY', 'minXmaxY', 'maxXmaxY']].values.tolist()

## clean up unnecessary columns
df_clean = df.drop(columns =['minX', 'minY', 'maxX', 'maxY', 'maxXmaxY', 'maxXminY', 'minXminY', 'minXmaxY', 'boundingBox'])

## Part 5: Create County GeoJSON

###  Step 7: Create geojson features

In [ ]:
# create_geojson_features 
def create_geojson_features(df):
    print('> Creating GeoJSON features...')
    features = []
    geojson = {
        'type': 'FeatureCollection',
        'features': features
    }
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Polygon', 
                'coordinates':[row['Coordinates']]
            },
            'properties': {
                'County': row['County'] + ' County',
                'State': row['State']
            }
        }

        features.append(feature)
    return geojson

data_geojson = create_geojson_features(df_clean)

###  Step 8: Generate geojson file

In [ ]:
with open(output, 'w') as txtfile:
    json.dump(data_geojson, txtfile)
print('> Creating GeoJSON file...')

## Part 6: Inspect bounding box map

In [ ]:
print('> Making map...')
## change the location here to zoom to the center
m = folium.Map(location = [42.3756, -93.6397], control_scale = True, zoom_start = 5)

## check if the indexmap geojson files can be rendered properly
folium.GeoJson(data_geojson, 
               tooltip = folium.GeoJsonTooltip(fields=('County', 'State'),
               aliases=('County', 'State')),
               show = True).add_to(m)
m